In [ ]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

In [ ]:
df = pd.read_csv("MU3D Codebook3.0.csv",encoding='cp1252')
df

,Accuracy,TruthProp,Transcription,Result,Truthful1,Trustworthy,Result2,Truthful
0,0.77,0.77,My best friend is a really nice person. Um. Sh...,0.5929,1,0.432,1.0249,1
1,0.40,0.60,She’s actually really two faced and not fun to...,0.2400,0,0.375,0.6150,0
2,0.77,0.77,So this specific person is actually just a rea...,0.5929,1,0.395,0.9879,1
3,0.58,0.42,This person is actually a really kind person. ...,0.2436,0,0.405,0.6486,0
4,0.59,0.59,My roommate is a really good person. She’s ver...,0.3481,0,0.436,0.7841,1
...,...,...,...,...,...,...,...,...
315,0.50,0.50,He’s a really good guy. Uh. He’s always cleani...,0.2500,0,0.420,0.6700,0
316,0.47,0.47,My roommate is a great guy. There is many grea...,0.2209,0,0.411,0.6319,0
317,0.45,0.55,"My roommate is insufferable. He is, uh, comple...",0.2475,0,0.427,0.6745,0
318,0.50,0.50,My cousin is a terrible person. She makes me r...,0.2500,0,0.432,0.6820,0


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df['Truthful'].value_counts()

0    215
1    105
Name: Truthful, dtype: int64

In [ ]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df['clean_text'] = df['Transcription'].apply(lambda x: finalpreprocess(x))
df.head()

,Accuracy,TruthProp,Transcription,Result,Truthful1,Trustworthy,Result2,Truthful,clean_text
0,0.77,0.77,My best friend is a really nice person. Um. Sh...,0.5929,1,0.432,1.0249,1,best friend really nice person um shes always ...
1,0.40,0.60,She’s actually really two faced and not fun to...,0.2400,0,0.375,0.6150,0,shes actually really two face fun around um sh...
2,0.77,0.77,So this specific person is actually just a rea...,0.5929,1,0.395,0.9879,1,specific person actually really mean negative ...
3,0.58,0.42,This person is actually a really kind person. ...,0.2436,0,0.405,0.6486,0,person actually really kind person many friend...
4,0.59,0.59,My roommate is a really good person. She’s ver...,0.3481,0,0.436,0.7841,1,roommate really good person shes sweet shes ho...


In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"],df["Truthful"],test_size=0.2,shuffle=True)

In [ ]:
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [ ]:
#gives less weightage to less important words?
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
def fit(self, X, y):
        return self
def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [ ]:
from sklearn import model_selection, naive_bayes, svm

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
#print(classification_report(y_test,y_predict))
#print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
print("SVM Accuracy Score -> ",accuracy_score(y_predict,y_test )*100)

SVM Accuracy Score ->  73.4375


In [ ]:
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.73      1.00      0.85        47
           1       0.00      0.00      0.00        17

    accuracy                           0.73        64
   macro avg       0.37      0.50      0.42        64
weighted avg       0.54      0.73      0.62        64

Confusion Matrix: [[47  0]
 [17  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=4, gamma=3)
SVM.fit(X_train_vectors_tfidf,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test_vectors_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM,y_test )*100)

SVM Accuracy Score ->  65.625


In [ ]:
print(classification_report(y_test,predictions_SVM))
print('Confusion Matrix:',confusion_matrix(y_test, predictions_SVM))

              precision    recall  f1-score   support

           0       0.71      0.89      0.79        47
           1       0.00      0.00      0.00        17

    accuracy                           0.66        64
   macro avg       0.36      0.45      0.40        64
weighted avg       0.52      0.66      0.58        64

Confusion Matrix: [[42  5]
 [17  0]]
